In [1]:
import pandas as pd

In [2]:
electeds = pd.read_excel('california-electeds.xlsx', sheetname=None)

/home/ryan/.virtualenvs/censusqueries/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [3]:
def readelecteds(sheet):
    df = electeds[sheet][['District','Name','Party']]
    df['Party'] = df['Party'].str.slice(stop=1)
    df['chamber'] = sheet[:-2]
    df['session'] = '20' + sheet[-2:]
    return df

In [4]:
districts = pd.concat([readelecteds(s) for s in electeds.keys()])

/home/ryan/.virtualenvs/censusqueries/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ryan/.virtualenvs/censusqueries/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/ryan/.virtualenvs/censusqueries/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [5]:
districts['District'] = districts['District'].astype(str).str.pad(width=3, fillchar='0')

In [6]:
districts.sample(5)

,District,Name,Party,chamber,session
4,005,Mike Thompson,D,congress,2017
21,022,Kevin Mullin,D,assembly,2019
29,030,Holly Mitchell,D,senate,2015
28,029,Mark Stone,D,assembly,2013
12,013,Susan Eggman,D,assembly,2015


In [7]:
districts.chamber.unique()

array(['senate', 'assembly', 'congress'], dtype=object)

In [8]:
blocks = pd.read_csv('blocks-uniquedistricts-xwalk.csv', dtype=str)

In [9]:
uniquedist = blocks[['legdist_c','legdist_a','legdist_s']].drop_duplicates().reset_index(drop=True)

In [10]:
uniquedist.head()

,legdist_c,legdist_a,legdist_s
0,013,015,009
1,013,018,009
2,015,018,009
3,015,020,010
4,015,020,009


In [11]:
def domerge(y):
    thisyear = districts[districts.session == y] 
    
    c = thisyear.loc[thisyear.chamber == 'congress', ['District','Party']]\
    .rename(columns={'District':'legdist_c','Party':'party_c'})
    
    a = thisyear.loc[thisyear.chamber == 'assembly', ['District','Party']]\
    .rename(columns={'District':'legdist_a','Party':'party_a'})
    
    s = thisyear.loc[thisyear.chamber == 'senate', ['District','Party']]\
    .rename(columns={'District':'legdist_s','Party':'party_s'})
        
    final = uniquedist.merge(c).merge(a).merge(s)
    final['year'] = y
    
    return final

In [12]:
yearly = {str(k): domerge(str(k)) for k in range(2013, 2021, 2)}

In [13]:
together = pd.concat(yearly.values())

In [14]:
together['district_code'] = 'c' + together.legdist_c + 'a' + together.legdist_a + 's' + together.legdist_s

In [15]:
together.head()

,legdist_c,legdist_a,legdist_s,party_c,party_a,party_s,year,district_code
0,013,015,009,D,D,D,2013,c013a015s009
1,011,015,009,D,D,D,2013,c011a015s009
2,005,015,009,D,D,D,2013,c005a015s009
3,013,018,009,D,D,D,2013,c013a018s009
4,015,018,009,D,D,D,2013,c015a018s009


In [16]:
def rbp(row):
    if row.party_c == 'D' and row.party_a == 'D' and row.party_s == 'D':
        return 'blue'
    if row.party_c == 'R' and row.party_a == 'R' and row.party_s == 'R':
        return 'red'
    return 'purple'

In [17]:
parties = together\
[['year','district_code','party_c','party_a','party_s']]

In [18]:
parties['color'] = parties.apply(rbp, axis=1)

/home/ryan/.virtualenvs/censusqueries/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
parties.head()

,year,district_code,party_c,party_a,party_s,color
0,2013,c013a015s009,D,D,D,blue
1,2013,c011a015s009,D,D,D,blue
2,2013,c005a015s009,D,D,D,blue
3,2013,c013a018s009,D,D,D,blue
4,2013,c015a018s009,D,D,D,blue


In [20]:
parties.to_csv('color-byuniquedistricts-byyear.csv', index=False)

In [21]:
byud = parties.pivot(index='district_code', columns='year', values='color')
byud.columns = ['color_13','color_15','color_17','color_19']
byud.reset_index(inplace=True)

In [22]:
byud.head()

,district_code,color_13,color_15,color_17,color_19
0,c001a001s001,red,red,red,red
1,c001a001s004,red,red,red,red
2,c001a003s004,red,red,red,red
3,c001a005s001,red,red,red,red
4,c001a006s001,red,red,red,red


In [23]:
len(byud)

389

In [24]:
byud.to_csv('color-byuniquedistricts-overyears.csv', index=False)

In [25]:
popcounts = pd.read_csv('blocks-uniquedistricts-xwalk.csv')\
.groupby('district_code')[['population']].sum()\
.reset_index()\
.merge(parties)\
.groupby(['color','year'])['population'].sum()\
.unstack(0)

In [26]:
popcounts

color,blue,purple,red
year,,,
2013,21222148,10551863,5479945
2015,21058683,8234764,7960509
2017,22072892,7794906,7386158
2019,24731942,8177075,4344939


In [27]:
popcounts.apply(lambda x: round(x / x.sum() * 100, 2), axis=1)

color,blue,purple,red
year,,,
2013,56.97,28.32,14.71
2015,56.53,22.10,21.37
2017,59.25,20.92,19.83
2019,66.39,21.95,11.66


In [28]:
popcounts.apply(lambda x: round(x / x.sum() * 100, 2), axis=1).to_dict()

{'blue': {'2013': 56.97, '2015': 56.53, '2017': 59.25, '2019': 66.39},
 'purple': {'2013': 28.32, '2015': 22.1, '2017': 20.92, '2019': 21.95},
 'red': {'2013': 14.71, '2015': 21.37, '2017': 19.83, '2019': 11.66}}

In [29]:
popcounts.to_dict()

{'blue': {'2013': 21222148,
  '2015': 21058683,
  '2017': 22072892,
  '2019': 24731942},
 'purple': {'2013': 10551863,
  '2015': 8234764,
  '2017': 7794906,
  '2019': 8177075},
 'red': {'2013': 5479945, '2015': 7960509, '2017': 7386158, '2019': 4344939}}